# CPSC322 Final Project -- UFO Sightings Dataset
### Sebastian Matthews and Ethan France

## 1. Introduction
The dataset that we have used for our project contains over 80,000 UFO sighting reports across the world, providing a variety of descriptions pertaining to each case(e.g. datetime, city, state, country, shape, duration in seconds and hours/min, comments regarding the report, date posted, and longitude and latitude), all sourced from [Kaggle](https://www.kaggle.com/datasets/NUFORC/ufo-sightings) and in a CSV format. The second dataset that we used was information regarding weather conditions during the day of the sighting, which we assembled from [Wunderground's Historical Weather Reports](https://www.wunderground.com/history) via a Selenium-based web scraping bot, which stored data an Excel file format and was later merged into our primary dataset. Finally, the third dataset used as an input for the Selenium bot was a CSV file that contained information regarding [ICAO codes](https://github.com/ip2location/ip2location-iata-icao/blob/master/iata-icao.csv), allowing the bot to input the location into the website and retrieve the weather data.

## 2. Data Analysis
The attributes that we utilized as class information were...
- The dew point average of a given day, labeled "dew-avg".
- The atmospheric pressure of a given day, labeled as "pressure-avg".
- The temperature average of a given day, labeled as "temp-avg".
- The average wind speed of a given day, labeled as "wind-avg".
- The humidity average of a given day, labeled as "humidity-avg".
- The total precipitation in inches for a given day, labeled as "precipitation-total".
All of these attributes were continuous numerical values, which assisted in predicting whether or not a UFO sighting will occur for a given day based on weather features and location in our binary classification scenario. The expected outcome for our predictions would then be stored in a "prediction" column(a boolean value) within the table, telling us whether or not a UFO sighting was present. 
![Pie Chart](Binary_Pie.png)
![Matrix](Binary_Matrix.png)
![Frequency Chart](Binary_Feature_Importance.png)

## 3. Classification Results

## 4. Classification Web App

## 5. Conclusion

## 6. Acknowledgements
Historic weather data sourced from [Wunderground](https://www.wunderground.com/history).
<br>
Project idea inspired by Bilal Ali Shah's [article on Medium](https://medium.com/@24020041/ufo-dataset-predicting-ufo-sightings-in-the-us-7539c95e75a8).